<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

### How can I analyze what 1000+ portfolio owners, i.e. large institutional investors, are doing in the housing market?

In this notebook, we will be examining 1000+ portfolio acquisitions, dispositions, and resale inventory. This notebook will work for any location in the US, whether it be a zip code, city, or metro area. To fine tune it for your location, modify the search criteria.

#### What will you create in this notebook?

**Institutional Activity Data for Download**

| Metro                                       | Acquisitions | Dispositions | Net | New Listings for Sale | New Rental Listings |
|---------------------------------------------|--------------|--------------|-----|-----------------------|---------------------|
| Las Vegas-Henderson-Paradise, NV            | 322          | 299          | 23  | 63                    | 539                 |
| Atlanta-Sandy Springs-Alpharetta, GA        | 189          | 95           | 94  | 274                   | 3151                |
| Phoenix-Mesa-Chandler, AZ                   | 134          | 105          | 29  | 108                   | 1450                |
| Jacksonville, FL                            | 119          | 108          | 11  | 71                    | 846                 |
| Charlotte-Concord-Gastonia, NC-SC           | 118          | 107          | 11  | 107                   | 1091                |
| Tampa-St. Petersburg-Clearwater, FL         | 113          | 90           | 23  | 115                   | 1138                |
| Dallas-Fort Worth-Arlington, TX             | 111          | 136          | -25 | 181                   | 1478                |
| Detroit-Warren-Dearborn, MI                 | 103          | 125          | -22 | 30                    | 15                  |
| Chicago-Naperville-Elgin, IL-IN-WI          | 99           | 50           | 49  | 35                    | 146                 |
| Indianapolis-Carmel-Anderson, IN            | 83           | 77           | 6   | 57                    | 534                 |

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/investor_analytics/institutional_activity.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs

In [3]:
# Environment setup
import os
import pandas as pd
from parcllabs import ParclLabsClient

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Markets

In [4]:
# Get top 100 CBSAs by population
markets = client.search.markets.retrieve(
    location_type='CBSA',
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    limit=100
)

markets.head()

,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market
0,2900187,USA,35620,None,"New York-Newark-Jersey City, Ny-Nj-Pa",None,None,CBSA,19908595,93610,0,1,1,1
1,2900078,USA,31080,None,"Los Angeles-Long Beach-Anaheim, Ca",None,None,CBSA,13111917,89105,0,1,1,1
2,2899845,USA,16980,None,"Chicago-Naperville-Elgin, Il-In-Wi",None,None,CBSA,9566955,85087,0,1,1,1
3,2899734,USA,19100,None,"Dallas-Fort Worth-Arlington, Tx",None,None,CBSA,7673379,83398,0,1,0,1
4,2899967,USA,26420,None,"Houston-The Woodlands-Sugar Land, Tx",None,None,CBSA,7142603,78061,0,1,0,0


### 3. Retrieve the Data

In [6]:
# get housing events for top 100 CBSA's for 
# portfolios of 1000 or more units
# get the most recent data from April

data = client.portfolio_metrics.sf_housing_event_counts.retrieve(
    parcl_ids=markets['parcl_id'].tolist(),
    portfolio_size='PORTFOLIO_1000_PLUS',
    limit=1
)

|████████████████████████████████████████| 100/100 [100%] in 7.6s (13.14/s) 


### 4. Prepare the Data

In [24]:
# join with market name for friendly display
df = data.merge(markets[['name', 'parcl_id']], on='parcl_id')
df.head()

,date,acquisitions,dispositions,new_listings_for_sale,new_rental_listings,parcl_id,portfolio_size,name
0,2024-05-01,17,14,12,1,2900187,PORTFOLIO_1000_PLUS,"New York-Newark-Jersey City, Ny-Nj-Pa"
1,2024-05-01,13,19,13,59,2900078,PORTFOLIO_1000_PLUS,"Los Angeles-Long Beach-Anaheim, Ca"
2,2024-05-01,99,50,35,146,2899845,PORTFOLIO_1000_PLUS,"Chicago-Naperville-Elgin, Il-In-Wi"
3,2024-05-01,111,136,181,1478,2899734,PORTFOLIO_1000_PLUS,"Dallas-Fort Worth-Arlington, Tx"
4,2024-05-01,83,65,114,822,2899967,PORTFOLIO_1000_PLUS,"Houston-The Woodlands-Sugar Land, Tx"


In [25]:
# capture the net acquisitions (are they net buyers or net sellers in a given market)
df['net'] = df['acquisitions'] - df['dispositions']
# format col names
df = df.rename(columns={
    'name': 'Metro',
    'acquisitions': 'Acquisitions',
    'dispositions': 'Dispositions',
    'net': 'Net',
    'new_listings_for_sale': 'New Listings for Sale',
    'new_rental_listings': 'New Rental Listings'
})

# sort on acquisitions to show the most acquiriing markets first
df = df.sort_values(by='Acquisitions', ascending=False)

In [41]:
# reorder columns for friendly display
df = df[['Metro', 'date', 'Acquisitions', 'Dispositions', 'Net', 'New Listings for Sale', 'New Rental Listings']]
df.head(10)

,Metro,date,Acquisitions,Dispositions,Net,New Listings for Sale,New Rental Listings
28,"Las Vegas-Henderson-Paradise, Nv",2024-05-01,322,299,23,63,539
8,"Atlanta-Sandy Springs-Alpharetta, Ga",2024-05-01,189,95,94,274,3151
10,"Phoenix-Mesa-Chandler, Az",2024-05-01,134,105,29,108,1450
38,"Jacksonville, Fl",2024-05-01,119,108,11,71,846
22,"Charlotte-Concord-Gastonia, Nc-Sc",2024-05-01,118,107,11,107,1091
17,"Tampa-St. Petersburg-Clearwater, Fl",2024-05-01,113,90,23,115,1138
3,"Dallas-Fort Worth-Arlington, Tx",2024-05-01,111,136,-25,181,1478
13,"Detroit-Warren-Dearborn, Mi",2024-05-01,103,125,-22,30,15
2,"Chicago-Naperville-Elgin, Il-In-Wi",2024-05-01,99,50,49,35,146
32,"Indianapolis-Carmel-Anderson, In",2024-05-01,83,77,6,57,534


### 5. Save the Data

In [ ]:
# save to csv file
df.to_csv('top_100_cbsas_institutional_sfh_events.csv', index=False)